In [ ]:
import pandas as pd
import numpy as np

# Membaca dataset
df = pd.read_csv('4. Paris Housing.csv')

# Memisahkan kolom numerik dan kategorik
numbers = df.select_dtypes(include=['number']).columns
categories = df.select_dtypes(exclude=['number']).columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Menghitung batas Quartile
Q1 = df[numbers].quantile(0.25)
Q3 = df[numbers].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Memfilter data (Hanya menyimpan data yang BUKAN outlier)
df_clean = df[~((df[numbers] < lower_bound) | (df[numbers] > upper_bound)).any(axis=1)]

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Mengubah 'Basic'/'Luxury' menjadi 0/1
le = LabelEncoder()
df_clean['category_encoded'] = le.fit_transform(df_clean['category'])

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled_df.columns
vif_data["VIF"] = [
    variance_inflation_factor(X_scaled_df.values, i)
    for i in range(X_scaled_df.shape[1])
]
print(vif_data)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Split Data (80% Train, 20% Test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 2. Scaling (Fit pada train, Transform pada test)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
    

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train_scaled, Y_train)
Y_pred = model.predict(X_test_scaled)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Menentukan kandidat alpha (dari 0.001 sampai 1000)
alphas = np.logspace(-3, 3, 20)
param_grid = {'alpha': alphas}

# Grid Search
grid = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid.fit(X_train_scaled, Y_train)

best_model = grid.best_estimator_
print(f"Alpha Terbaik: {grid.best_params_}")

In [ ]:
from sklearn.linear_model import Lasso

# Proses sama seperti Ridge
grid_lasso = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_lasso.fit(X_train_scaled, Y_train)

best_lasso = grid_lasso.best_estimator_